In [12]:
import geopandas as gpd
import ee
import geemap as gee
import datetime as dt
import numpy as np

# import matplotlib.pyplot as plt
# ee.Authenticate()
gee.ee_initialize()

# Setup

In [13]:
begTime = dt.date.today()
today = ee.Date(begTime.strftime("%Y-%m-%d"))

# Read

In [14]:
df_uf = gpd.read_file('../data/vector/uf_sp.shp')
df_br =  gpd.read_file('../data/vector/br.shp')
df_silvicultura = gpd.read_file('../data/vector/silvicultura_2021_clean50.shp')

In [15]:
silv = ee.FeatureCollection("projects/ee-lucaspontesm/assets/silvicultura_2021_clean50");

In [16]:
# aoi = ee.Geometry.Polygon(list(df_uf[df_uf['SIGLA_UF']=='SP'].geometry.exterior[0].coords))
# br = ee.Geometry.Polygon(list(df_br.geometry.exterior[0].coords))
# silvicultura = ee.Geometry.MultiPolygon([list(x.coords) for x in df_silvicultura.geometry.exterior])

In [17]:
lulc = ee.Image("projects/ee-lucaspontesm/assets/MAPBIOMAS/mapbiomas-brazil-collection-71-saopaulo-2021")
gfs_collection = ee.ImageCollection('NOAA/GFS0P25')
imerg_collection = ee.ImageCollection("NASA/GPM_L3/IMERG_V06")
viirs_collection = ee.ImageCollection("NOAA/VIIRS/001/VNP14A1")
dem = ee.Image("CGIAR/SRTM90_V4")

In [18]:
aoi = ee.FeatureCollection("projects/ee-lucaspontesm/assets/uf_sp").geometry(); 

# UDF

In [19]:
flame_cte = {1: ["Forest", 2],
            2: ["Natural Forest", 2],
            3: ["Forest Formation", 2],
            4: ["Savanna Formation", 2.4],
            5: ["Magrove", 1.5],
            6: ["Áreas Naturales Inundables - Leñosas (Bosque Inundable)",1.5],
            9: ["Forest Plantation", 4],
            10: ["Non Forest Natural Formation", 2],
            11: ["Wetland",1.5],
            12: ["Grassland (Pastizal, Formación Herbácea)", 6],
            13: ["Other Non Forest Natural Formation", 2],
            14: ["Farming", 4],
            15: ["Pasture", 6],
            18: ["Agriculture",4],
            19: ["Temporary Crops (Herbaceas - Agricultura)",4],
            20: ["Sugar Cane",4],
            21: ["Mosaic of Agriculture and Pasture",5],
            # 22: ["Non vegetated area",None],
            # 23: ["Beach and Dune",None],
            # 24: ["Urban Infrastructure", None],
            # 25: ["Other Non Vegetated Area", None],
            # 26: ["Water", None],
            # 27: "Non Observed",
            # 29: ["Rocky outcrop", None],
            # 30: ["Mining", None],
            # 31: ["Aquaculture", None],
            # 32: "Salt flat",
            # 33: ["River, Lake and Ocean", None],
            # 34: ["Glacier", None],
            35: ["Oil Palm",4],
            36: ["Perennial Crops",3],
            # 37: ["Artificial Water Body", None],
            # 38: ["Water Reservoirs", None],
            39: ["Soy Beans",4],
            40: ["Rice",4],
            41: ["Mosaic of Crops",4],
            46: ['Coffe',4],
            47: ['Citrus',4],
            48: ['Other Perennial Crops', 4],
            49: ['Wooded Sandbank Vegetation', 3],
            50: ['Herbaceous Sandbank Vegetation', 3],
            62: ["Cotton", 4],
            # 0: "Non Observed"
            }

# pre processing

In [20]:
# https://developers.google.com/earth-engine/apidocs/ee-image-remap
# A list of pixel values to replace.
fromList = list(flame_cte.keys())

# A corresponding list of replacement values (1 becomes 1, 2 becomes 2, etc).
toList = [x[1] for x in flame_cte.values()]

# Replace pixel values in the image. If the image is multi-band, only the
# remapped band will be returned. The returned band name is "remapped".
# Input image properties are retained in the output image.
flamability = lulc.remap(**{
  'from': fromList,
  'to': toList,
  'defaultValue': None,
  'bandName': 'b1'
})

In [21]:
# flamability.reduceRegion(**{
#                     'reducer': ee.Reducer.min(),
#                     'scale': 11000,
#     }).getInfo()

# IMERG rainfall

In [22]:
# iteração para pegar a imagem de chuva mais recente
p1 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))

while len(p1.getInfo()['bands']) == 0:
      begTime = (begTime - dt.timedelta(days=1))
      today = ee.Date(begTime.strftime("%Y-%m-%d"))

      p1 = (imerg_collection
            .filterDate((begTime - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
                        begTime.strftime("%Y-%m-%d"))
            .select('precipitationCal')
            .sum()
            .clip(aoi))

In [23]:
p2 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=2)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))

p3 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=3)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))

p4 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=4)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))

p5 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=5)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p10 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=10)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p15 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=15)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p30 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=30)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p60 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=60)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p90 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=90)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))
p120 = (imerg_collection
      .filterDate((begTime - dt.timedelta(days=120)).strftime("%Y-%m-%d"),
                   begTime.strftime("%Y-%m-%d"))
        .select('precipitationCal')
        .sum()
        .clip(aoi))

In [24]:
# p120.reduceRegion(**{
#                     'reducer': ee.Reducer.max(), 
#                     'geometry': aoi, 
#                     'scale': 11000,
#     }).getInfo()

In [25]:
fp1 = p1.expression('exp(-0.14 *p1)',
                    {'p1': p1.select('precipitationCal')})

fp2 = p2.expression('exp(-0.07 *(pf-pi))',
                    {'pi': p1.select('precipitationCal'),
                     'pf': p2.select('precipitationCal')})

fp3 = p3.expression('exp(c *(pf-pi))',
                    {'c':-0.04,
                    'pi': p2.select('precipitationCal'),
                    'pf': p3.select('precipitationCal')})

fp4 = p4.expression('exp(c *(pf-pi))',
                    {'c':-0.03,
                    'pi': p3.select('precipitationCal'),
                    'pf': p4.select('precipitationCal')})

fp5 = p5.expression('exp(c *(pf-pi))',
                    {'c':-0.02,
                    'pi': p4.select('precipitationCal'),
                    'pf': p5.select('precipitationCal')})

fp6a10 = p10.expression('exp(c *(pf-pi))',
                    {'c':-0.01,
                    'pi': p5.select('precipitationCal'),
                    'pf': p10.select('precipitationCal')})

fp11a15 = p15.expression('exp(c *(pf-pi))',
                    {'c':-0.008,
                    'pi': p10.select('precipitationCal'),
                    'pf': p15.select('precipitationCal')})

fp16a30 = p30.expression('exp(c *(pf-pi))',
                    {'c':-0.004,
                    'pi': p15.select('precipitationCal'),
                    'pf': p30.select('precipitationCal')})

fp31a60 = p60.expression('exp(c *(pf-pi))',
                    {'c':-0.002,
                    'pi': p30.select('precipitationCal'),
                    'pf': p60.select('precipitationCal')})

fp61a90 = p90.expression('exp(c *(pf-pi))',
                    {'c':-0.001,
                    'pi': p60.select('precipitationCal'),
                    'pf': p90.select('precipitationCal')})

fp91a120 = p120.expression('exp(c *(pf-pi))',
                    {'c':-0.0007,
                    'pi': p90.select('precipitationCal'),
                    'pf': p120.select('precipitationCal')})

In [26]:
# Calcula os dias de secura (PSE)
pse = (fp1
        .multiply(fp2)
        .multiply(fp3)
        .multiply(fp4)
        .multiply(fp5)
        .multiply(fp6a10)
        .multiply(fp11a15)
        .multiply(fp16a30)
        .multiply(fp31a60)
        .multiply(fp61a90)
        .multiply(fp91a120)
        .multiply(105)
)


In [27]:
# pse.reduceRegion(**{
#                     'reducer': ee.Reducer.min(), 
#                     'geometry': aoi, 
#                     'scale': 11000,
#     }).getInfo()

In [28]:
# calcula o Risco Basico de fogo (Rb)

rb = pse.expression('0.8*(1+sin((((a*pse)-90))*(3.14/180)))/2',
                    {'a':flamability.select('remapped'),
                    'pse': pse.select('constant')
                    })

In [29]:
# rb.reduceRegion(**{
#                     'reducer': ee.Reducer.max(), 
#                     'geometry': aoi, 
#                     'scale': 11000,
#     }).getInfo()

# GFS

In [30]:
# Map = gee.Map()
# temperatureAboveGround = gfs_collection.filter(ee.Filter.date('2023-07-17', '2023-07-18')).select('temperature_2m_above_ground').filterBounds(br).first()
# palette =  ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
# temperature_vis = {'min': -40.0, 'max': 35.0, 'palette': palette}

# Map.setCenter(-40, -10, 4)
# Map.addLayer(temperatureAboveGround, temperature_vis, 'Temperature Above Ground')
# Map

In [32]:
####### REVER #####################
temperature = gfs_collection.filter(ee.Filter.date(begTime.strftime("%Y-%m-%d"))).select('temperature_2m_above_ground').filterBounds(aoi).max()
relative_humidity = gfs_collection.filter(ee.Filter.date(begTime.strftime("%Y-%m-%d"))).select('relative_humidity_2m_above_ground').filterBounds(aoi).min()

In [33]:
ft = temperature.expression('(Tmax*0.02)+0.4',
                            {'Tmax': temperature})

In [34]:
fu = relative_humidity.expression('(UR * -0.006)+1.3',
                                {'UR':relative_humidity})

# Risco de Fogo Observado

In [35]:
rf = rb.multiply(ft).multiply(fu)

In [36]:
flat = rf.expression('1+abs(lat)*0.003', {'lat':rf.pixelLonLat().select('latitude')})
felv = dem.expression('1+alt*0.00003', {'alt':dem.select('elevation')})

In [37]:
rfb = rf.multiply(flat).multiply(felv)

# Classificacao do risco de fogo

In [38]:
# rf_class = (ee.Image(1)
#       .where(rf.select('constant').lte(0.15), 1)
#       .where((rf.select('constant').gt(0.15)) and (rf.select('constant').lte(0.40)), 2)
#       .where((rf.select('constant').gt(0.40)) and (rf.select('constant').lte(0.70)), 3)
#       .where((rf.select('constant').gt(0.70)) and (rf.select('constant').lte(0.95)), 4)
#       .where(rf.select('constant').gt(0.95), 5)).clip(aoi)

#  Focos de incêndio

In [39]:
if viirs_collection.filterDate(today.advance(-3, 'day'), today).size().getInfo() == 0:
    rfb_ajustado= rfb
else:   
  focos = (viirs_collection
    .filterDate(today.advance(-3, 'day'), today)
    .select('FireMask').max().gt(6).clip(aoi)
    .remap(**{
          'from': [0,1],
          'to': [1,2],
          'defaultValue': None,
          'bandName': 'FireMask'
          })
  )

  rfb_ajustado = rfb.multiply(focos)

# Display the result.


In [42]:
Map = gee.Map()

palette = [
  '000096','0064ff', '00b4ff', '33db80', '9beb4a',
  'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000'
  ]

precipitationVis = {'min': 0, 'max': 100, 'palette': palette}
temperatureVis = {'min': 0, 'max': 40, 'palette': palette}
fpVis = {'min': 0, 'max': 1, 'palette': palette}
vis_classe_fogo = {'min': 0, 'max': 1, 'palette': ['green', 'lime', 'yellow', 'red', 'maroon']}

# Map.addLayer(p1, precipitationVis, 'P (mm)')
Map.addLayer(pse, precipitationVis, 'Dias de Secura')
# Map.addLayer(rf, fpVis, 'Risco Fogo Observado')
# Map.addLayer(rb, fpVis, 'Risco Básico de Fogo')
# Map.addLayer(rfb_ajustado, vis_classe_fogo, 'Risco de Fogo Observado')
# Map.addLayer(focos)
Map.addLayer(silv, {}, 'Áreas de Silvicultura', shown= False)
Map.setCenter(-46, -24, 6)  
Map

Map(center=[-24, -46], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…